In [ ]:
import sys

# to allow imports from outside the tsp_ml/datasets/ package
sys.path.insert(0, "/home/pimenta/tsp_ml/tsp_ml")
import pandas as pd
from os import listdir

from paths import  get_predictions_folder_path
import torch
import networkx as nx
from torch_geometric.utils.convert import to_networkx

In [ ]:
# get filepath
step="test_small"
# trained_model_name = "2022_09_19_23h55_GreedyPathsModel"
trained_model_name = "2022_09_29_17h31_GreedyCyclesModel"
predictions_dir = get_predictions_folder_path(
    dataset_name="KEP",
    step=step,
    trained_model_name=trained_model_name,
)
predicted_instances_dir = predictions_dir / "predicted_instances"
# filename = listdir(predicted_instances_dir)[0]
instance_id = "5507bd9a3a1d054fe277c8f88ad2e4dc"
filename=(instance_id +"_pred.pt")
filepath = predicted_instances_dir / filename

# load predicted instance
predicted_instance = torch.load(filepath)
# cast to nx.Graph
nx_kep_graph = to_networkx(
    data=predicted_instance,
    node_attrs=["type"],
    edge_attrs=["edge_weights", "pred", "scores"],

)
nx_kep_graph

In [ ]:
labels = nx.get_edge_attributes(G=nx_kep_graph, name="edge_weights")
labels


In [ ]:
# color nodes according to their type
color_map = []
for node_id in nx_kep_graph:
    node_type = nx_kep_graph.nodes[node_id]["type"]
    if node_type == "NDD":
        color_map.append('lightgreen')
    elif node_type == 'PDP':
        color_map.append('lightblue')
    elif node_type == "P":
        color_map.append('red')
    else:
        raise ValueError(f'Type {node_type} is not valid.')

edges = nx_kep_graph.edges()
edge_widths = [nx_kep_graph[u][v]['edge_weights']*3 for u,v in edges]
nx.draw(G=nx_kep_graph, node_color=color_map, with_labels=True, width=edge_widths)



In [ ]:
# get predicted edges
predicted_edges = []
for (edge_src, edge_dst, edge_features) in nx_kep_graph.edges(data=True):
    if edge_features['pred'] == 1:
        predicted_edges.append((edge_src, edge_dst))

In [ ]:
# plot graph
posisions_dict = nx.spring_layout(nx_kep_graph)
nx.draw(G=nx_kep_graph, pos=posisions_dict, node_color=color_map, with_labels=True, width=edge_widths)
# plot predicted route in red with weights as labels
nx.draw_networkx_edges(G=nx_kep_graph,pos=posisions_dict,edgelist=predicted_edges, edge_color ='r', width=edge_widths)
labels = nx.get_edge_attributes(G=nx_kep_graph, name="edge_weights")
labels = {k: f"{v:.2f}" for k, v in labels.items()} # limit digits
nx.draw_networkx_edge_labels(G=nx_kep_graph,pos=posisions_dict,edge_labels=labels, font_size=8)
# optional: save plot
# import matplotlib.pyplot as plt
# plot_filename = trained_model_name + + ".png" # TODO
# plt.savefig('plotgraph.png', dpi=300, bbox_inches='tight')
# plt.show()


In [ ]:
predicted_instance

In [ ]:
print(predicted_instance.edge_index)
print(predicted_instance.scores)
print(predicted_instance.pred)
print(predicted_instance.pred * predicted_instance.edge_index)
print(predicted_instance.pred.view(-1, 1) * predicted_instance.scores)